# Azure Computer Vision 4 (Florence)

## Background removal with Azure Computer Vision 4 and Gradio

![Image](florence.jpg)

**The Image Analysis** service can divide images into multiple segments or regions to help the user identify different objects or parts of the image. **Background removal** creates an alpha matte that separates the foreground object from the background in an image.


This feature provides two possible outputs based on the customer's needs:

- The **foreground object** of the image without the background. This edited image shows the foreground object and makes the background transparent, allowing the foreground to be placed on a new background.
- An **alpha matte** that shows the opacity of the detected foreground object. This matte can be used to separate the foreground object from the background for further processing.

This service is currently in preview, and the API may change in the future.

https://learn.microsoft.com/en-us/azure/cognitive-services/computer-vision/concept-background-removal

Azure Computer Vision updates: https://learn.microsoft.com/en-us/azure/cognitive-services/computer-vision/whats-new#march-2023
<br><br>
> Serge Retkowsky | Microsoft | https://github.com/retkowsky | 3rd of May, 2023

![Image](backgroundremovalimage.jpg)

In [1]:
import gradio as gr
import io
import os
import requests
import sys

from dotenv import load_dotenv
from PIL import Image

In [2]:
sys.version

'3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]'

In [3]:
load_dotenv("azure.env")

azure_cv_key = os.getenv("azure_cv_key")
azure_cv_endpoint = os.getenv("azure_cv_endpoint")

In [4]:
def remove_background_from_image(image, endpoint=azure_cv_endpoint, key=azure_cv_key):
    """
    Remove background from image using Azure Computer Vision
    """
    url = (
        endpoint
        + "/computervision/imageanalysis:segment?api-version=2023-02-01-preview&mode="
    )
    remove_background_url = url + "backgroundRemoval"  # For removing the background
    get_mask_object_url = url + "foregroundMatting"  # Mask of the object

    headers = {
        "Content-type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": key,
    }

    with io.BytesIO() as output:
        image.save(output, format="JPEG")
        image_data = output.getvalue()

    with requests.Session() as session:
        try:
            r_wobackground = session.post(
                remove_background_url, data=image_data, headers=headers
            )
            r_mask = session.post(get_mask_object_url, data=image_data, headers=headers)
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)

    wo_background_image = Image.open(io.BytesIO(r_wobackground.content))
    mask_image = Image.open(io.BytesIO(r_mask.content))

    return wo_background_image, mask_image

In [5]:
logo = "https://github.com/retkowsky/images/blob/master/logoazureCV.png?raw=true"
image = "<center> <img src= {} width=50px></center>".format(logo)

samples = [
    "images/tomato.jpg",
    "images/fruits.jpg",
    "images/drinks.jpg",
    "images/car.jpg",
]

header = "Background removal with Azure Computer Vision"

# Define Gradio interface
inputs = gr.Image(type="pil", label="Input Image")
outputs = [
    gr.Image(label="Image without background", type="pil"),
    gr.Image(label="Mask", type="pil"),
]

# Gradio Webapp
webapp = gr.Interface(
    remove_background_from_image,
    inputs,
    outputs,
    title=header,
    description=image,
    examples=samples,
    theme="gstaff/sketch",  # https://huggingface.co/spaces/gradio/theme-gallery
)

# Running the webapp
webapp.launch(share=True)

themes/theme_schema@0.0.1.json:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

C:\Users\xinyuwei\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\xinyuwei\.cache\huggingface\hub\spaces--gstaff--sketch. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Created dataset file at: .gradio\flagged\dataset1.csv
